In [1]:
import gensim
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
model = gensim.models.Word2Vec.load("/home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model")
model.init_sims(replace=True)

2017-05-17 00:35:20,751 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model
2017-05-17 00:35:23,101 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model.wv.* with mmap=None
2017-05-17 00:35:23,102 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model.wv.syn0.npy with mmap=None
2017-05-17 00:35:24,963 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model.syn1neg.npy with mmap=None
2017-05-17 00:35:30,035 : INFO : setting ignored attribute syn0norm to None
2017-05-17 00:35:30,394 : INFO : setting ignored attribute cum_table to None
2017-05-17 00:35:30,399 : INFO : loaded /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model
2017-05-17 00:35:32,512 : INFO : precomputing L2-norms of word weight vectors


In [4]:
num_features = 400

### Verify model with personality

In [5]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [6]:
df.conscientiousness_m.value_counts()

0    523
1    516
Name: conscientiousness_m, dtype: int64

In [7]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

### Evaluation code

In [8]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(2)
    target_names = ['no', 'yes']
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [9]:
def evaluate_prediction(predictions, target, title="Confusion matrix"):
    print('accuracy %s' % accuracy_score(target, predictions))
    cm = confusion_matrix(target, predictions)
    print('confusion matrix\n %s' % cm)
    print('(row=expected, col=predicted)')
    
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plot_confusion_matrix(cm_normalized, title + ' Normalized')

In [10]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['conscientiousness_m']
    evaluate_prediction(predictions, target)

### Word2Vec model training

In [11]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [12]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [13]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [14]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-05-17 00:37:48,194 : WARNING : cannot compute similarity with no input [u'FOREIGN']
2017-05-17 00:37:48,906 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:49,366 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:49,374 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:50,257 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:50,589 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:50,962 : WARNING : cannot compute similarity with no input [u'Infelizmente', u'Verdade', u'EMOTION*']
2017-05-17 00:37:51,091 : WARNING : cannot compute similarity with no input [u'FOREIGN', u'FOREIGN', u'FOREIGN']
2017-05-17 00:37:51,111 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:52,054 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:52,173 : WARNING : cannot compute similarity with no input []
2017-05-17 00:37:52,433 : WARNING : cannot comp

CPU times: user 5.4 s, sys: 41.8 ms, total: 5.45 s
Wall time: 5.6 s


In [15]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [16]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['conscientiousness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.1s finished


CPU times: user 5.95 s, sys: 354 ms, total: 6.31 s
Wall time: 30.4 s


In [17]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['conscientiousness_m']

In [18]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,93,74
1,60,85


In [19]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.61      0.56      0.58       167
          1       0.53      0.59      0.56       145

avg / total       0.57      0.57      0.57       312



In [20]:
accuracy_score(yt, yp)

0.57051282051282048